# API INMET

A [API INMET](https://portal.inmet.gov.br/manual) possui diversos endpoints. Neste estudo, o objetivo é coletar e tratar séries temporais de estações meteorológicas.

## Lista de estações

Para recuperar a lista de estações automáticas: https://apitempo.inmet.gov.br/estacoes/T

## Dados por estação

Recuperar dados horários referentes a estação automática ou manual:
- https://apitempo.inmet.gov.br/estacao/{data_inicial}/{data_final}/{cod_estacao}
- https://apitempo.inmet.gov.br/estacao/2019-10-23/2019-10-23/A301
- Sendo data no formato AAAA-MM-DD (ex: 2019-10-23)

In [1]:
using HTTP
using CSV
using JSON3
import Dates
using DataFrames

In [6]:
# configurações

# os dados serão coletados posto a posto em um intervalo de datas
df = Dates.DateFormat("y-m-d")
dataInicial = Dates.Date("2021-01-01", df)
dataFinal = Dates.Date("2021-01-02", df)

# cria um diretório para salvar os dados obtidos
diretorioPersistencia = "dados_estacoes"
Base.mkpath(diretorioPersistencia);

In [2]:
"""
Executa uma chamada GET simples a uma API.

# Examples
```jldoctest
julia> url = "https://apitempo.inmet.gov.br/estacao/2021-01-01/2021-01-02/A756"
julia> resposta = chamarAPI(url)
```
"""
function chamarAPI(url)
    # chamarAPI("https://apitempo.inmet.gov.br/estacao//2021-01-01/2021-01-02/A756")[1]
    # chamarAPI("https://apitempo.inmet.gov.br/estacao//2021-01-01/2021-01-02/A225")[1]
    resposta = HTTP.get(url)
    respostaString = String(resposta.body)
    return JSON3.read(respostaString, allow_inf=true)
end;

In [3]:
listaEstacoesAutomaticasJSON = chamarAPI("https://apitempo.inmet.gov.br/estacoes/T")
listaEstacoesAutomaticas = DataFrame(listaEstacoesAutomaticasJSON);

# filtrando apenas estações operantes
listaEstacoesAutomaticasOperantes = listaEstacoesAutomaticas[
    listaEstacoesAutomaticas[:, "CD_SITUACAO"] .== "Operante", :];

In [7]:
function coletarDadosEstacao(
        codigoEstacao::String,
        dataInicial::Dates.Date,
        dataFinal::Dates.Date
)::DataFrame
    # coletarDadosEstacao("A225", dataInicial, dataFinal)
    
    urlBaseDadosEstacoes = "https://apitempo.inmet.gov.br/estacao"
    urlQueryDadosEstacoes = "$(dataInicial)/$(dataFinal)/$(codigoEstacao)"
    urlCompleta = "$(urlBaseDadosEstacoes)/$(urlQueryDadosEstacoes)"
    
    # println(urlCompleta)

    serieEstacaoJSON = chamarAPI(urlCompleta)
    serieEstacao = DataFrame(serieEstacaoJSON)
    
    # substitui valores 'nothing' por 'missing'
    for coluna in names(serieEstacao)
        serieEstacao[!, coluna] = replace(
            serieEstacao[!, coluna], nothing => missing
        )
    end
    
    # se o dataframe gerador for vazio, a função irá parar
    if size(dropmissing(serieEstacao))[1] == 0
        error("não há dados disponíveis para a estação '$(codigoEstacao)'.")
    end
    
    return serieEstacao
end;

In [8]:
codigosEstacoes = listaEstacoesAutomaticasOperantes[:, "CD_ESTACAO"]

for codigoEstacao in codigosEstacoes
    try
        serieEstacao = coletarDadosEstacao(codigoEstacao, dataInicial, dataFinal)
        CSV.write("$(diretorioPersistencia)/$(codigoEstacao).csv", serieEstacao)
        println("Dados da estação '$(codigoEstacao)' obtidos.")
    catch e
        println("Não foi possível processar os dados de '$(codigoEstacao)': $(e)")
    end
    
    Base.sleep(1)
end

Dados da estação 'A756' obtidos.
Dados da estação 'A045' obtidos.
Dados da estação 'A549' obtidos.
Dados da estação 'A826' obtidos.
Dados da estação 'A615' obtidos.
Não foi possível processar os dados de 'A053': ErrorException("não há dados disponíveis para a estação 'A053'.")
Dados da estação 'A508' obtidos.
Dados da estação 'A909' obtidos.
Dados da estação 'A024' obtidos.
Não foi possível processar os dados de 'A934': ErrorException("não há dados disponíveis para a estação 'A934'.")
Dados da estação 'A336' obtidos.
Dados da estação 'A434' obtidos.
Dados da estação 'S701' obtidos.
Dados da estação 'A340' obtidos.
Dados da estação 'A409' obtidos.
Dados da estação 'A566' obtidos.
Não foi possível processar os dados de 'A013': ErrorException("não há dados disponíveis para a estação 'A013'.")
Não foi possível processar os dados de 'A054': ErrorException("não há dados disponíveis para a estação 'A054'.")
Dados da estação 'S702' obtidos.
Dados da estação 'A867' obtidos.
Dados da estação 'A5